# Importing packages

In [ ]:
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import io, json

### Beep code

In [ ]:
import winsound
duration = 500  # milliseconds
freq = 240  # Hz

# Coin List and related data

In [ ]:
os.chdir('D:\Dropbox\Dropbox\ADITYA-CRYPTO\CODES\OUTPUT\SOCIAL_DATA')

def coin_list():
    url = 'https://min-api.cryptocompare.com/data/all/coinlist'
    page = requests.get(url)
    data = page.json()['Data']
    return data

data = coin_list()

In [ ]:
import io, json
with io.open('coin_list.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(data, ensure_ascii= True))

df = json.load(open("coin_list.json", "r"))
df = pd.DataFrame.from_dict(data, orient="index")

coin_list = df['Symbol']
df.to_csv (r'coin_list.csv', index = None)

In [ ]:
df1 = []
i=0
for coin in coin_list:
  try:
    i+=1
    print(i)
    url = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym={}&tsym=USD&limit=2000&api_key=19329eecd83f2fc87469e0e49f5e5c4154a347643d645f416a12f305e95d7e65'.format(coin)
    # url = str(url)
    print(url)
    page = requests.get(url)
    data = page.json()['Data']

    # print(data)
    data = data['Data']

    with io.open('hist_data.json', 'w', encoding='utf-8') as f:
      f.write(json.dumps(data, ensure_ascii= False))

    df = pd.read_json (r'hist_data.json')
    # print(df)
    df['time'] = [datetime.fromtimestamp(x) for x in df['time']]
    df['time'] = pd.to_datetime(df['time']).dt.strftime('%d-%m-%y')
    df['ticker'] = str(coin)
    df1.append(df)

  except:
    pass

df_final = pd.concat(df1)
df_final.to_hdf(r'hist_data.hd5', key='df',  index = None)

winsound.Beep(freq, duration)


In [ ]:
# os.chdir('D:\Dropbox\Dropbox\ADITYA-CRYPTO\CODES\BTC_DATA')

def BTC_data():
    url = 'https://min-api.cryptocompare.com/data/blockchain/histo/day?fsym=x42&limit=2000&api_key=19329eecd83f2fc87469e0e49f5e5c4154a347643d645f416a12f305e95d7e65'
    page = requests.get(url)
    data = page.json()['Data']
    return data

data = BTC_data()['Data']

# Social Data - Extraction

In [ ]:
def social_data():
    url = 'https://min-api.cryptocompare.com/data/social/coin/histo/day?&limit=2000&api_key=19329eecd83f2fc87469e0e49f5e5c4154a347643d645f416a12f305e95d7e65'
    page = requests.get(url)
    data = page.json()['Data']
    return data

data = social_data()

In [ ]:
import io, json
with io.open('social_data.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(data, ensure_ascii= False))

 
df = pd.read_json (r'social_data.json')
df['time'] = [datetime.fromtimestamp(x) for x in df['time']]
df['time'] = pd.to_datetime(df['time']).dt.strftime('%d-%m-%y')

df.drop(np.where(~(df.loc[:, df.columns != 'time']).any(axis=1))[0], axis=0, inplace=True)
df['time_plot'] = pd.to_datetime(df['time']).dt.strftime('%m')
df.to_csv (r'social_data.csv', index = None)

#  Social Data - Graphs

In [ ]:
# Level 
for column in df:
    if column =='time':
        continue
    plot = df.plot(x='time', y=column , legend = None)
    plot.set_title(column)
    plot.tick_params(axis='x', rotation=90)
    plt.savefig(r'plots/level/'+str(column)+'.png',dpi=600,bbox_inches='tight')
    plt.clf()

In [ ]:
# % change
df1= df.drop(['time', 'time_plot'], axis=1)
change = df1.div(df1.shift(1))
change = change-1
df3 = df['time']

In [ ]:
df2 = pd.concat([change, df3], axis=1, join="inner")
 
for column in df:
    if column =='time':
        continue
    plot = df2.plot(x='time', y=column , legend = None)
    plot.set_title(column)
    plot.tick_params(axis='x', rotation=90)
    plt.savefig(r'plots/change/'+str(column)+'.png',dpi=600,bbox_inches='tight')
    plt.clf()



# BTC Data - Extraction

In [ ]:
os.chdir('D:\Dropbox\Dropbox\ADITYA-CRYPTO\CODES\BTC_DATA')

def BTC_data():
    url = 'https://min-api.cryptocompare.com/data/blockchain/histo/day?fsym=BTC&limit=2000&api_key=19329eecd83f2fc87469e0e49f5e5c4154a347643d645f416a12f305e95d7e65'
    page = requests.get(url)
    data = page.json()['Data']
    return data

data = BTC_data()['Data']

In [ ]:
import io, json
with io.open('btc_data.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(data, ensure_ascii= False))

 
df = pd.read_json (r'btc_data.json')
df['time'] = [datetime.fromtimestamp(x) for x in df['time']]
df['time'] = pd.to_datetime(df['time']).dt.strftime('%d-%m-%y')

df.drop(np.where(~(df.loc[:, df.columns != 'time']).any(axis=1))[0], axis=0, inplace=True)
df.to_csv (r'btc_data.csv', index = None)

# BTC - Graphs

In [ ]:
# Level 
first_column = df.pop('transaction_count_all_time')
df.insert(5, "transaction_count_all_time", first_column)

first_column = df.pop('time')
df.insert(6, "time", first_column)

df_plot = df.iloc[:, 6:]

for column in df_plot:
    if column =='time':
        continue
    plot = df_plot.plot(x='time', y=column , legend = None)
    plot.set_title(column)
    plot.tick_params(axis='x', rotation=90)
    plt.savefig(r'plots/level/'+str(column)+'.png',dpi=600,bbox_inches='tight')
    plt.clf()

In [ ]:
# % change
df1= df_plot.drop(['time'], axis=1)
change = df1.div(df1.shift(1))
change = change-1
df3 = df_plot['time']

In [ ]:
df2 = pd.concat([change, df3], axis=1, join="inner")
 
for column in df2:
    if column =='time':
        continue
    plot = df2.plot(x='time', y=column , legend = None)
    plot.set_title(column)
    plot.tick_params(axis='x', rotation=90)
    plt.savefig(r'plots/change/'+str(column)+'.png',dpi=600,bbox_inches='tight')
    plt.clf()



# Google Trends Data

In [ ]:
import pandas as pd                        
from pytrends.request import TrendReq
from pytrends import dailydata
import time

### Scaled Data

In [ ]:
kw = ["bitcoin", "crypto", "cryptocurrency", "cryptocurrencies", "ethereum", "blockchain"]
 
df_final=[]
for x in kw:
    df2=[]
    for i in range(2016, 2022):
        if i ==2021:

            df = dailydata.get_daily_data(x, start_year=2021, start_mon=1, stop_year=2021, stop_mon=10)[str(x)]
            df2.append(df)
            # time.sleep(1)
        else:

            df = dailydata.get_daily_data(x, start_year=i, start_mon=1, stop_year=i, stop_mon=12)[str(x)]
            df2.append(df)
            # time.sleep(1)
    df2 = pd.concat(df2) 
    df_final.append(df2)
    df_final1 = pd.concat(df_final, axis=1)
df_final1.to_csv (r'TRENDS_data_scaled.csv')

### Unscaled Data

In [ ]:

kw = ["bitcoin", "crypto", "cryptocurrency", "cryptocurrencies", "ethereum", "blockchain"]
 
df_final=[]
for x in kw:
    df2=[]
    for i in range(2016, 2022):
        if i ==2021:
            df = dailydata.get_daily_data(x, start_year=2021, start_mon=1, stop_year=2021, stop_mon=10)[str(x)+'_unscaled']
            df2.append(df)
        else:
            df = dailydata.get_daily_data(x, start_year=i, start_mon=1, stop_year=i, stop_mon=12)[str(x)+'_unscaled']
            df2.append(df)
    df2 = pd.concat(df2) 
    df_final.append(df2)
    df_final1 = pd.concat(df_final, axis=1)
df_final1.to_csv (r'TRENDS_data_unscaled.csv')

# Glassnode Data

In [ ]:
import pandas as pd                        
from pytrends.request import TrendReq
from pytrends import dailydata
import time
import requests

In [ ]:
def glassnode():
    API_KEY = '21h23ZDbBh3fJnve63rXhA4FvEG'
    res =  requests.get('https://api.glassnode.com/v1/metrics/endpoints', params={'a': 'BTC', 'api_key': API_KEY})
    data = pd.read_json(res.text, convert_dates=['t'])
    return data

metrics = glassnode()

In [ ]:
metrics = metrics['Path']
metrics

In [ ]:
# create dataframe
assets = ["BTC","ETH","LTC"]

for asst in assets:
    df1=[]
    df= []
    for path in metrics:
        try:
            API_KEY = '21h23ZDbBh3fJnve63rXhA4FvEG'
            path2 = 'https://api.glassnode.com'+str(path)
            res =  requests.get('https://api.glassnode.com'+str(path), params={'a': str(asst), 'api_key': API_KEY})
            path2 = path2.rsplit('/', 1)[-1]
            data = pd.read_json(res.text, convert_dates=['t'])
            data = data.rename(columns={"v": str(path2)})
            df.append(data)
            time.sleep(5)  
        except:
            pass
    df1 = pd.concat(df, axis=1) 
    df1.to_excel(str(asst)+".xlsx") 

In [ ]:
def glassnode():
    API_KEY = '21h23ZDbBh3fJnve63rXhA4FvEG'
    res =  requests.get('https://api.glassnode.com/v1/metrics/assets', params={'a': 'BTC', 'api_key': API_KEY})
    data = pd.read_json(res.text, convert_dates=['t'])
    return data

assets = glassnode()

# Sanbase

In [ ]:
import san
import pandas as pd
import time

In [ ]:
slugs = san.get("projects/all")
slugs[slugs["name"]=="Ripple"]

In [ ]:
san.available_metrics_for_slug(str(slug))

In [ ]:
coins = ["bitcoin","ripple","litecoin", "ethereum"]


for slug in coins: 
    df1= []
    for i in range(49,349,50):
        avail = san.available_metrics_for_slug(str(slug))[i-49:i]
        try:
            for metric in avail:
                df = san.get(str(metric)+"/"+str(slug), from_date="2010-01-01", to_date="2021-10-31", interval="1d")
                df = df.rename(columns={"value": str(metric)})
                df1.append(df)
                
        except:
            pass 
        df_final = pd.concat(df1, axis=1)
        time.sleep(3)

    df_final.reset_index(inplace=True)
    df_final["datetime"] = df_final["datetime"].apply(str)
    df_final.to_excel(str(slug)+'.xlsx')
    time.sleep(60)


# CoinMetrics

In [ ]:
import coinmetrics 
import pandas as pd

In [ ]:
coins = ["btc","eth","xrp", "ltc"]


for cns in coins: 
    df1= []
    avail = cm.get_available_data_types_for_asset(str(cns))
    try:
        for metric in avail:
            df = cm.get_asset_data_for_time_range(str(cns), str(metric), "2010-01-01", "2021-10-31")['series']
            values= pd.DataFrame (df, columns = ['values'])
            values = values.values.tolist()
            values = [x[0] for x in values]
            values= pd.DataFrame(values)
            df2 = pd.DataFrame (df, columns = ['time'])
            df3 = [df2, values]
            df4 = pd.concat(df3, axis=1)
            df4 = df4.rename(columns={0: str(metric)})
            df1.append(df4)
            
    except:
        pass
    df_final = pd.concat(df1, axis=1)
    df_final.reset_index(drop=True, inplace=True)
    df_final = df_final.loc[:,~df_final.columns.duplicated()]
    df_final.to_excel(str(cns)+'.xlsx')
# time.sleep(60)
